In [ ]:
import numpy as np
import pandas as pd
import glob
import math
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import scipy.io as sio
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# onehot encoding of kineme sequence
def onehot_encoding(ks, nKineme):
    #print(ks)
    onehot_encoded = list()
    for k in ks:
        #print(k)
        vec = [0 for _ in range(nKineme)]
        vec[k-1] = 1
        onehot_encoded.append(vec)
        #print("Vector")
        #print(vec)
    return onehot_encoded
def ks_encoding(ks, nKineme):
    # ks is a numpy ndarray
    m, n = ks.shape #m=92, n=29
    #print(m, n)
    ks = ks.tolist() #converted to list
    encoded_features = np.asarray(
        [np.asarray(onehot_encoding(ks[i], nKineme)) for i in range(m)]
    )
    return encoded_features

In [ ]:
def model_formation(seqLen):
    Model_Kineme = Sequential()
    Model_Kineme.add(LSTM(20,activation="tanh",dropout=0.2,recurrent_dropout=0.0,input_shape=(seqLen, 16),return_sequences =False ))
    Model_Kineme.add(Dense(units = 1, activation='linear'))
    opt = keras.optimizers.Adam(learning_rate=0.01)
    Model_Kineme.compile(optimizer = opt, loss = 'mean_absolute_error')
    Model_Kineme.summary()

    #AU Model
    Model_AU = Sequential()
    Model_AU.add(LSTM(20,activation="tanh",dropout=0.2,recurrent_dropout=0.0,input_shape=(seqLen, 17),return_sequences =False ))
    Model_AU.add(Dense(units = 1, activation='linear'))
    opt = keras.optimizers.Adam(learning_rate=0.01)
    Model_AU.compile(optimizer = opt, loss = 'mean_absolute_error')
    Model_AU.summary()

    #Audio Model
    Model_Audio = Sequential()
    Model_Audio.add(LSTM(20,activation="tanh",dropout=0.2,recurrent_dropout=0.0,input_shape=(seqLen, 23),return_sequences =False ))
    Model_Audio.add(Dense(units = 1, activation='linear'))
    opt = keras.optimizers.Adam(learning_rate=0.01)
    Model_Audio.compile(optimizer = opt, loss = 'mean_absolute_error')
    Model_Audio.summary()

    return Model_Kineme, Model_AU, Model_Audio

In [ ]:
#Train----FICS, val---FICS Test----MIT
def model_call(Model_Kineme, Model_AU, Model_Audio):
    Audio_train_data_path = 'path' #path for fics train audio data array
    train_label_path = 'path' #path for fics train labels array
    Audio_test_data_path = 'path' #path for mit test audio data array
    test_label_path = 'path' #path for mit test labels array
    Audio_val_data_path = 'path' #path for fics validation audio data array
    val_label_path = 'path' #path for fics validation labels array

    AU_train_data_path = 'path' #path for fics train AU data array
    AU_test_data_path = 'path' #path for MIT test AU data array
    AU_val_data_path = 'path' #path for fics validation AU data array

    Kineme_train_data_path = 'path' #path for fics train Kineme data array
    Kineme_test_data_path = 'path' #path for MIT test Kineme data array
    Kineme_val_data_path = 'path' #path for fics validation Kineme data array


    X_train_audio = np.load(Audio_train_data_path)
    y_train = np.load(train_label_path)[:,1]
    y_train = y_train.astype(float)
    y_train = np.around(y_train,3)

    X_val_audio = np.load(Audio_val_data_path)
    y_val = np.load(val_label_path)[:,1]
    y_val = y_val.astype(float)
    y_val = np.around(y_val,3)

    X_test_audio = np.load(Audio_test_data_path)
    y_test = np.load(test_label_path)
    y_test = np.around(y_test,3)

    X_train_au = np.load(AU_train_data_path)
    X_val_au = np.load(AU_val_data_path)
    X_test_au = np.load(AU_test_data_path)

    X_train_kineme = np.load(Kineme_train_data_path)
    X_val_kineme = np.load(Kineme_val_data_path)
    X_test_kineme = np.load(Kineme_test_data_path)

    scaler = preprocessing.StandardScaler().fit(X_train_audio)
    X_train = scaler.transform(X_train_audio)
    X_val = scaler.transform(X_val_audio)
    X_test = scaler.transform(X_test_audio)


    train_features,train_labels = X_train,y_train
    test_features, test_labels = X_test, y_test
    val_features, val_labels = X_val, y_val

    train_audio = train_features.reshape((train_features.shape[0], seqLen, 23))
    test_audio = test_features.reshape((test_features.shape[0], seqLen, 23))
    val_audio = val_features.reshape((val_features.shape[0], seqLen,23 ))

    train_au = X_train_au.reshape((X_train_au.shape[0], seqLen, 17))
    test_au = X_test_au.reshape((X_test_au.shape[0], seqLen, 17))
    val_au = X_val_au.reshape((X_val_au.shape[0], seqLen,17))

    train_kineme = ks_encoding(X_train_kineme, 16)
    test_kineme = ks_encoding(X_test_kineme, 16)
    val_kineme = ks_encoding(X_val_kineme, 16)

    print(train_audio.shape, test_audio.shape, val_audio.shape)
    print(train_au.shape, test_au.shape, val_au.shape)
    print(train_kineme.shape, test_kineme.shape, val_kineme.shape)


    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    #Model Fitting
    zero_bias_history_kineme = Model_Kineme.fit(train_kineme, train_labels, epochs = 250, batch_size = 100, validation_data = (val_kineme, val_labels))
    print('Kineme Training Done!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    zero_bias_history_AU = Model_AU.fit(train_au, train_labels, epochs = 250, batch_size = 100, validation_data = (val_au, val_labels))
    print('AU Training Done!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    zero_bias_history_Audio = Model_Audio.fit(train_audio, train_labels, epochs = 250, batch_size = 100, validation_data = (val_audio, val_labels))
    print('Audio Training Done!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')

    train_pred_kineme = Model_Kineme.predict(train_kineme)
    train_pred_au = Model_AU.predict(train_au)
    train_pred_audio = Model_Audio.predict(train_audio)

    test_pred_kineme = Model_Kineme.predict(test_kineme)
    test_pred_au = Model_AU.predict(test_au)
    test_pred_audio = Model_Audio.predict(test_audio)

    return train_pred_kineme, train_pred_au, train_pred_audio, test_pred_kineme, test_pred_au, test_pred_audio, train_labels, test_labels

In [ ]:
seqLen = 14
# x = [0.33,0,0.5,0.5,0.25,0.5,0.25]
# y = [0.33,0.5,0,0.5,0.5,0.25,0.25]
# z = [0.33,0.5,0.5,0,0.25,0.25,0.5]

x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.6, 0.6, 0.6, 0.6, 0.7, 0.7, 0.7, 0.8, 0.8, 0.8, 0.9, 0.9, 1.0]
y = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.0, 0.1, 0.2, 0.4, 0.0, 0.1, 0.3, 0.0, 0.1, 0.2, 0.0, 0.1, 0.0]
z = [1.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.0, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.0, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0, 0.4, 0.3, 0.2, 0.0, 0.3, 0.2, 0.0, 0.2, 0.1, 0.0, 0.1, 0.0, 0.0]


tr_acc_list = []
te_acc_list = []
tr_pcc_list = []
te_pcc_list = []

Model_Kineme, Model_AU, Model_Audio = model_formation(seqLen)
# model_call(Model_Kineme, Model_AU, Model_Audio)


train_pred_kineme, train_pred_au, train_pred_audio, test_pred_kineme, test_pred_au, test_pred_audio,train_labels, test_labels = model_call(Model_Kineme, Model_AU, Model_Audio)

for i in range(len(x)):
    print(i)
    #print(x[i],y[i],z[i])


    final_train_pred = x[i]*train_pred_kineme + y[i]*train_pred_au + z[i]*train_pred_audio
    y_pred_train = np.around(final_train_pred,3)

    final_test_pred = x[i]*test_pred_kineme + y[i]*test_pred_au + z[i]*test_pred_audio
    # y_pred_test = final_model.predict([test_kinemes, test_aus,test_audio_data])
    y_pred_test = np.around(final_test_pred,3)

    print(train_labels.shape, y_pred_train.shape)
    print(test_labels.shape, y_pred_test.shape)

    train_mae = mean_absolute_error(train_labels, y_pred_train) ##mean squarred train error
    test_mae = mean_absolute_error(test_labels, y_pred_test) #mean squarred test error
    y_train = train_labels.reshape(-1,1)
    b = np.corrcoef(y_train.T,y_pred_train.T)
    train_PCC = b[0][1]
    w = test_labels.reshape(-1,1)
    a = np.corrcoef(w.T,y_pred_test.T)
    test_PCC = a[0][1]
    tr_acc = 1 - train_mae
    te_acc = 1-test_mae


    tr_acc_list.append(round(np.array(tr_acc).mean(),3))
    te_acc_list.append(round(np.array(te_acc).mean(),3))
    tr_pcc_list.append(round(np.array(train_PCC).mean(),3))
    te_pcc_list.append(round(np.array(test_PCC).mean(),3))

Final_mat = np.vstack((np.asarray(x),np.asarray(y), np.asarray(z), np.asarray(tr_acc_list),
                       np.asarray(te_acc_list), np.asarray(tr_pcc_list), np.asarray(te_pcc_list)))
Final_mat = Final_mat.T

path = '' #path to save csv file containing results for all values of x,y and z.
pd.DataFrame(Final_mat).to_csv(path)